## 3. Implementando os Simuladores para o Caso Exponencial

Cenário $\lambda = 1$ e $\mu  = 2$

Descrição: Considere uma fila M/M/1, em que chegadas ocorrem segundo um fluxo Poisson com taxa 1 chegadas/segundo e serviços são tais que cada serviço é distribuído exponencialmente com média 1/2 segundo. Implemente dois simuladores.

In [1]:
import random

In [22]:
def simulate(arrival_rate, service_rate, max_steps=1000):
    total_arrivals = total_departures = total_waiting_time = simulation_time = queue_length = 0
    arrivals_queue = []

    for i in range(max_steps):
        sample_arrival = random.expovariate(arrival_rate)
        sample_departure = random.expovariate(service_rate)

        if queue_length == 0 or sample_arrival < sample_departure:
            # Arrival
            simulation_time += sample_arrival
            total_arrivals += 1
            queue_length += 1
            if queue_length > 1:
                arrivals_queue.append(simulation_time)
        else:
            # Departure
            simulation_time += sample_departure
            total_departures += 1
            queue_length -= 1
            if len(arrivals_queue) > 0:
                total_waiting_time += simulation_time - arrivals_queue.pop(0)

    average_waiting_time = total_waiting_time / total_departures
    average_clients_in_system = total_arrivals / simulation_time

    return {
        "average_waiting_time": average_waiting_time,
        "average_clients_in_system": average_clients_in_system
    }
    print("Average waiting time: ", average_waiting_time)
    print("Average clients in system: ", average_clients_in_system)

In [23]:
simulate(1, 2, 1000)

{'average_waiting_time': 0.49929843845209476,
 'average_clients_in_system': 0.9917779324943828}